In [11]:
# CELL 1: Load and prepare data (SAME AS BEFORE)
import pandas as pd
import numpy as np

# Load the data
train_df = pd.read_csv("../../cleaned_dataset/split_data/model_selection/train.csv")
test_df = pd.read_csv("../../cleaned_dataset/split_data/model_selection/test.csv")

# Get common labels
train_labels = list(train_df['room'].unique())
test_labels = list(test_df['room'].unique())
common_labels = list(set(train_labels) & set(test_labels))

# Filter to common labels
train_df = train_df[train_df['room'].isin(common_labels)].reset_index(drop=True)
test_df = test_df[test_df['room'].isin(common_labels)].reset_index(drop=True)

# Sort by timestamp
train_df = train_df.sort_values('timestamp').reset_index(drop=True)
test_df = test_df.sort_values('timestamp').reset_index(drop=True)

print(f"Train set: {len(train_df)} windows, {train_df['room'].nunique()} rooms")
print(f"Test set: {len(test_df)} windows, {test_df['room'].nunique()} rooms")

Train set: 920927 windows, 16 rooms
Test set: 141425 windows, 16 rooms


In [12]:
# CELL 2: Create 23-dim beacon vectors WITH time_delta feature
def create_beacon_vectors_with_time_delta(df):
    """
    For each window, create:
    - 23-dim beacon count vector
    - time_delta (seconds since previous window)
    """
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    vectors = []
    prev_timestamp = None
    
    for timestamp, group in df.groupby('timestamp'):
        # Calculate time delta
        if prev_timestamp is None:
            time_delta = 0.0
        else:
            time_delta = (timestamp - prev_timestamp).total_seconds()
        
        # Count beacon frequencies
        beacon_counts = group['mac address'].value_counts()
        total_readings = len(group)
        
        # Create 23-dim vector
        vector = [0.0] * 23
        for beacon_id, count in beacon_counts.items():
            if 1 <= beacon_id <= 23:
                vector[beacon_id - 1] = count / total_readings
        
        vectors.append({
            'timestamp': timestamp,
            'room': group['room'].iloc[0],
            'beacon_vector': vector,
            'time_delta': time_delta
        })
        
        prev_timestamp = timestamp
    
    return pd.DataFrame(vectors)

# Create vectors
print("Creating beacon vectors with time_delta for train set...")
train_vectors = create_beacon_vectors_with_time_delta(train_df)
print(f"✓ Train vectors: {len(train_vectors)} windows")

print("\nCreating beacon vectors with time_delta for test set...")
test_vectors = create_beacon_vectors_with_time_delta(test_df)
print(f"✓ Test vectors: {len(test_vectors)} windows")

# Show examples
print("\nExample with time deltas:")
print(train_vectors[['timestamp', 'room', 'time_delta']].head(10))

Creating beacon vectors with time_delta for train set...
✓ Train vectors: 15052 windows

Creating beacon vectors with time_delta for test set...
✓ Test vectors: 5889 windows

Example with time deltas:
                  timestamp       room  time_delta
0 2023-04-10 14:21:46+09:00    kitchen         0.0
1 2023-04-10 14:21:47+09:00    kitchen         1.0
2 2023-04-10 14:21:48+09:00    kitchen         1.0
3 2023-04-10 14:21:49+09:00    kitchen         1.0
4 2023-04-10 14:21:50+09:00    kitchen         1.0
5 2023-04-10 14:21:57+09:00  cafeteria         7.0
6 2023-04-10 14:21:58+09:00  cafeteria         1.0
7 2023-04-10 14:21:59+09:00  cafeteria         1.0
8 2023-04-10 14:22:06+09:00  cafeteria         7.0
9 2023-04-10 14:22:07+09:00  cafeteria         1.0


In [13]:
# CELL 3: Create sliding window sequences (sequence-to-sequence)
def create_sliding_sequences(vector_df, window_size=100, stride=50):
    """
    Create overlapping sequences with sliding window approach.
    Each sequence contains window_size consecutive windows.
    
    Returns:
    - X: List of sequences, each sequence is (window_size, 24) 
         where 24 = 23 beacons + 1 time_delta
    - y: List of label sequences, each is (window_size,) room labels
    """
    X_sequences = []
    y_sequences = []
    
    # Get total number of windows
    n_windows = len(vector_df)
    
    # Slide window across data
    for start_idx in range(0, n_windows - window_size + 1, stride):
        end_idx = start_idx + window_size
        
        sequence_data = vector_df.iloc[start_idx:end_idx]
        
        # Extract features: beacon_vector (23-dim) + time_delta (1-dim) = 24-dim
        X_seq = []
        y_seq = []
        
        for _, row in sequence_data.iterrows():
            # Combine beacon vector with time_delta
            features = row['beacon_vector'] + [row['time_delta']]
            X_seq.append(features)
            y_seq.append(row['room'])
        
        X_sequences.append(np.array(X_seq))
        y_sequences.append(y_seq)
    
    return X_sequences, y_sequences

# Create sequences
window_size = 100  # 100-second windows
stride = 50        # 50% overlap

print(f"Creating sliding window sequences (window_size={window_size}, stride={stride})...")
X_train_seq, y_train_seq = create_sliding_sequences(train_vectors, window_size, stride)
X_test_seq, y_test_seq = create_sliding_sequences(test_vectors, window_size, stride)

print(f"\n✓ Train sequences: {len(X_train_seq)}")
print(f"✓ Test sequences: {len(X_test_seq)}")
print(f"\nEach sequence shape: {X_train_seq[0].shape} (window_size, num_features)")
print(f"Features: 23 beacons + 1 time_delta = 24 features")

# Show example
print(f"\nExample sequence (first 5 windows):")
for i in range(5):
    print(f"  Window {i}: time_delta={X_train_seq[0][i, -1]:.1f}s, room={y_train_seq[0][i]}")

Creating sliding window sequences (window_size=100, stride=50)...

✓ Train sequences: 300
✓ Test sequences: 116

Each sequence shape: (100, 24) (window_size, num_features)
Features: 23 beacons + 1 time_delta = 24 features

Example sequence (first 5 windows):
  Window 0: time_delta=0.0s, room=kitchen
  Window 1: time_delta=1.0s, room=kitchen
  Window 2: time_delta=1.0s, room=kitchen
  Window 3: time_delta=1.0s, room=kitchen
  Window 4: time_delta=1.0s, room=kitchen


In [14]:
# CELL 4: Encode labels for sequence-to-sequence
from sklearn.preprocessing import LabelEncoder

# Fit label encoder on all unique rooms
all_rooms = list(set(train_df['room'].unique()) | set(test_df['room'].unique()))
label_encoder = LabelEncoder()
label_encoder.fit(all_rooms)

num_classes = len(label_encoder.classes_)
print(f"Number of classes: {num_classes}")
print(f"Classes: {label_encoder.classes_}")

# Encode sequences (each window in sequence gets encoded)
def encode_sequence_labels(y_sequences, label_encoder):
    """Encode labels for each window in each sequence"""
    encoded_sequences = []
    for y_seq in y_sequences:
        encoded_seq = label_encoder.transform(y_seq)
        encoded_sequences.append(encoded_seq)
    return encoded_sequences

y_train_encoded = encode_sequence_labels(y_train_seq, label_encoder)
y_test_encoded = encode_sequence_labels(y_test_seq, label_encoder)

print(f"\n✓ Encoded train sequences: {len(y_train_encoded)}")
print(f"✓ Encoded test sequences: {len(y_test_encoded)}")
print(f"\nExample encoded sequence (first 5): {y_train_encoded[0][:5]}")

Number of classes: 16
Classes: ['501' '502' '506' '511' '512' '513' '517' '518' '520' '522' '523'
 'cafeteria' 'cleaning' 'hallway' 'kitchen' 'nurse station']

✓ Encoded train sequences: 300
✓ Encoded test sequences: 116

Example encoded sequence (first 5): [14 14 14 14 14]


In [15]:
# CELL 5: Convert to numpy arrays and one-hot encode
from tensorflow.keras.utils import to_categorical

# Convert to numpy arrays
X_train = np.array(X_train_seq)
X_test = np.array(X_test_seq)

# One-hot encode each window in each sequence
y_train_onehot = np.array([to_categorical(seq, num_classes=num_classes) for seq in y_train_encoded])
y_test_onehot = np.array([to_categorical(seq, num_classes=num_classes) for seq in y_test_encoded])

print(f"X_train shape: {X_train.shape}")
print(f"  (num_sequences, window_size, num_features)")
print(f"\ny_train shape: {y_train_onehot.shape}")
print(f"  (num_sequences, window_size, num_classes)")
print(f"\nX_test shape: {X_test.shape}")
print(f"y_test shape: {y_test_onehot.shape}")

X_train shape: (300, 100, 24)
  (num_sequences, window_size, num_features)

y_train shape: (300, 100, 16)
  (num_sequences, window_size, num_classes)

X_test shape: (116, 100, 24)
y_test shape: (116, 100, 16)


In [16]:
# CELL 6: Build sequence-to-sequence LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed

model = Sequential([
    # Input: (window_size, 24 features)
    
    # LSTM layers - return_sequences=True for sequence-to-sequence
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    
    # TimeDistributed applies Dense layer to each time step independently
    TimeDistributed(Dense(32, activation='relu')),
    Dropout(0.2),
    
    # Output layer: predict class at each time step
    TimeDistributed(Dense(num_classes, activation='softmax'))
])

# Compile
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Sequence-to-Sequence LSTM Model:")
model.summary()

Sequence-to-Sequence LSTM Model:


c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 100, 128)       │        78,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 100, 32)        │         2,080 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 100, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 100, 16)        │           528 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 130,352 (509.19 KB)

 Trainable params: 130,352 (509.19 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# CELL 7: Custom Macro F1 callback for sequence-to-sequence
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
from sklearn.metrics import f1_score

class SequenceToSequenceMacroF1Callback(Callback):
    """
    Monitor macro F1 at the WINDOW level (not sequence level)
    Flatten all predictions and labels across all sequences
    """
    def __init__(self, validation_data, patience=10):
        super().__init__()
        self.X_val, self.y_val = validation_data
        self.patience = patience
        self.best_macro_f1 = 0
        self.wait = 0
        self.best_weights = None
        
    def on_epoch_end(self, epoch, logs=None):
        # Predict on validation set
        y_pred_probs = self.model.predict(self.X_val, verbose=0)
        
        # Flatten: (num_sequences, window_size, num_classes) -> (total_windows, num_classes)
        y_pred_flat = y_pred_probs.reshape(-1, y_pred_probs.shape[-1])
        y_true_flat = self.y_val.reshape(-1, self.y_val.shape[-1])
        
        # Get class predictions
        y_pred_classes = np.argmax(y_pred_flat, axis=1)
        y_true_classes = np.argmax(y_true_flat, axis=1)
        
        # Calculate macro F1
        macro_f1 = f1_score(y_true_classes, y_pred_classes, average='macro', zero_division=0)
        
        print(f"\nEpoch {epoch + 1}: val_macro_f1 = {macro_f1:.4f}")
        
        # Check if best
        if macro_f1 > self.best_macro_f1:
            self.best_macro_f1 = macro_f1
            self.wait = 0
            self.best_weights = self.model.get_weights()
            print(f"  → New best macro F1! Saving weights.")
        else:
            self.wait += 1
            print(f"  → No improvement (patience: {self.wait}/{self.patience})")
            
            if self.wait >= self.patience:
                print(f"\n⚠️ Early stopping! Restoring best weights (macro F1 = {self.best_macro_f1:.4f})")
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

In [18]:
# CELL 8: Split train into train/val and compute class weights
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Split for validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train_onehot,
    test_size=0.2,
    random_state=42
)

print(f"Train split: {X_train_split.shape[0]} sequences")
print(f"Val split: {X_val_split.shape[0]} sequences")

# Compute class weights on flattened labels
y_train_flat = y_train_split.reshape(-1, y_train_split.shape[-1])
y_train_classes_flat = np.argmax(y_train_flat, axis=1)

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train_classes_flat),
    y=y_train_classes_flat
)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

print("\nClass weights:")
for i, weight in class_weight_dict.items():
    print(f"  {label_encoder.classes_[i]}: {weight:.2f}")

Train split: 240 sequences
Val split: 60 sequences

Class weights:
  501: 15.00
  502: 166.67
  506: 20.83
  511: 9.15
  512: 5.30
  513: 100.00
  517: 57.69
  518: 300.00
  520: 14.71
  522: 4.27
  523: 3.98
  cafeteria: 0.28
  cleaning: 9.49
  hallway: 1.73
  kitchen: 0.32
  nurse station: 0.13


In [19]:
# CELL 9: Train the model (FIXED - no class_weight for seq2seq)
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
from sklearn.metrics import f1_score

# Recreate the callback class
class SequenceToSequenceMacroF1Callback(Callback):
    """
    Monitor macro F1 at the WINDOW level (not sequence level)
    Flatten all predictions and labels across all sequences
    """
    def __init__(self, validation_data, patience=10):
        super().__init__()
        self.X_val, self.y_val = validation_data
        self.patience = patience
        self.best_macro_f1 = 0
        self.wait = 0
        self.best_weights = None
        
    def on_epoch_end(self, epoch, logs=None):
        # Predict on validation set
        y_pred_probs = self.model.predict(self.X_val, verbose=0)
        
        # Flatten: (num_sequences, window_size, num_classes) -> (total_windows, num_classes)
        y_pred_flat = y_pred_probs.reshape(-1, y_pred_probs.shape[-1])
        y_true_flat = self.y_val.reshape(-1, self.y_val.shape[-1])
        
        # Get class predictions
        y_pred_classes = np.argmax(y_pred_flat, axis=1)
        y_true_classes = np.argmax(y_true_flat, axis=1)
        
        # Calculate macro F1
        macro_f1 = f1_score(y_true_classes, y_pred_classes, average='macro', zero_division=0)
        
        print(f"\nEpoch {epoch + 1}: val_macro_f1 = {macro_f1:.4f}")
        
        # Check if best
        if macro_f1 > self.best_macro_f1:
            self.best_macro_f1 = macro_f1
            self.wait = 0
            self.best_weights = self.model.get_weights()
            print(f"  → New best macro F1! Saving weights.")
        else:
            self.wait += 1
            print(f"  → No improvement (patience: {self.wait}/{self.patience})")
            
            if self.wait >= self.patience:
                print(f"\n⚠️ Early stopping! Restoring best weights (macro F1 = {self.best_macro_f1:.4f})")
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

# Initialize callbacks
macro_f1_callback = SequenceToSequenceMacroF1Callback(
    validation_data=(X_val_split, y_val_split),
    patience=10
)

reduce_lr = ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

print("=" * 80)
print("Training Sequence-to-Sequence LSTM (Option 3)...")
print("=" * 80)
print("\nNote: class_weight not supported for sequence-to-sequence models")
print("Class imbalance will be handled through macro F1 optimization instead\n")

history = model.fit(
    X_train_split, y_train_split,
    validation_data=(X_val_split, y_val_split),
    epochs=50,
    batch_size=16,  # Adjust if memory issues
    # NO class_weight parameter - doesn't work with seq2seq!
    callbacks=[macro_f1_callback, reduce_lr],
    verbose=1
)

print(f"\n✓ Training complete!")
print(f"✓ Best validation macro F1: {macro_f1_callback.best_macro_f1:.4f}")

Training Sequence-to-Sequence LSTM (Option 3)...

Note: class_weight not supported for sequence-to-sequence models
Class imbalance will be handled through macro F1 optimization instead



Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.3648 - loss: 2.3358
Epoch 1: val_macro_f1 = 0.0442
  → New best macro F1! Saving weights.
15/15 ━━━━━━━━━━━━━━━━━━━━ 11s 371ms/step - accuracy: 0.4173 - loss: 2.0652 - val_accuracy: 0.4940 - val_loss: 1.4939 - learning_rate: 0.0010
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.5271 - loss: 1.5170
Epoch 2: val_macro_f1 = 0.0958
  → New best macro F1! Saving weights.
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.5361 - loss: 1.4888 - val_accuracy: 0.6325 - val_loss: 1.2383 - learning_rate: 0.0010
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5964 - loss: 1.3874
Epoch 3: val_macro_f1 = 0.0973
  → New best macro F1! Saving weights.
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - accuracy: 0.5895 - loss: 1.3461 - val_accuracy: 0.6278 - val_loss: 1.1633 - learning_rate: 0.0010
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6103 - loss: 1.3075
Epoch 4: val_macro_f1 = 0.11

In [20]:
# CELL 10: Evaluate on test set
from sklearn.metrics import classification_report

# Predict on test set
print("Predicting on test set...")
y_pred_probs = model.predict(X_test, verbose=1)

# Flatten predictions and labels
y_pred_flat = y_pred_probs.reshape(-1, y_pred_probs.shape[-1])
y_test_flat = y_test_onehot.reshape(-1, y_test_onehot.shape[-1])

y_pred_classes = np.argmax(y_pred_flat, axis=1)
y_true_classes = np.argmax(y_test_flat, axis=1)

# Decode to room names
y_pred = label_encoder.inverse_transform(y_pred_classes)
y_true = label_encoder.inverse_transform(y_true_classes)

# Calculate macro F1
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

print("\n" + "=" * 80)
print("SEQUENCE-TO-SEQUENCE LSTM EVALUATION (OPTION 3)")
print("=" * 80)
print(f"\n🎯 MACRO F1 SCORE: {macro_f1:.4f}")
print(f"   Option 2 (room-segmented) was: 0.37")
print(f"   XGBoost baseline was: 0.30")

if macro_f1 > 0.37:
    improvement = (macro_f1 - 0.37) / 0.37 * 100
    print(f"   ✓ Improvement over Option 2: +{improvement:.1f}%")
else:
    decline = (0.37 - macro_f1) / 0.37 * 100
    print(f"   ✗ Decline: -{decline:.1f}%")

print("\n" + "=" * 80)
print("PER-CLASS F1 SCORES")
print("=" * 80)

# Get per-class F1
report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

class_f1_scores = []
for class_name in sorted(set(y_true)):
    if class_name in report:
        f1 = report[class_name]['f1-score']
        support = report[class_name]['support']
        class_f1_scores.append((class_name, f1, support))

class_f1_scores.sort(key=lambda x: -x[1])

print(f"\n{'Room':<20s} {'F1 Score':<12s} {'Support':<10s}")
print("-" * 80)
for room, f1, support in class_f1_scores:
    print(f"{room:<20s} {f1:<12.4f} {int(support):<10d}")

print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Average F1 (macro): {macro_f1:.4f}")
print(f"Best room: {class_f1_scores[0][0]} (F1={class_f1_scores[0][1]:.4f})")
print(f"Worst room: {class_f1_scores[-1][0]} (F1={class_f1_scores[-1][1]:.4f})")
print(f"Rooms with F1 > 0.5: {sum(1 for _, f1, _ in class_f1_scores if f1 > 0.5)}/{len(class_f1_scores)}")
print(f"Rooms with F1 < 0.2: {sum(1 for _, f1, _ in class_f1_scores if f1 < 0.2)}/{len(class_f1_scores)}")

Predicting on test set...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

SEQUENCE-TO-SEQUENCE LSTM EVALUATION (OPTION 3)

🎯 MACRO F1 SCORE: 0.2702
   Option 2 (room-segmented) was: 0.37
   XGBoost baseline was: 0.30
   ✗ Decline: -27.0%

PER-CLASS F1 SCORES

Room                 F1 Score     Support   
--------------------------------------------------------------------------------
kitchen              0.7246       3306      
512                  0.6339       266       
nurse station        0.6108       2502      
523                  0.5961       220       
520                  0.4199       380       
506                  0.3926       194       
cafeteria            0.3917       2446      
522                  0.2500       34        
501                  0.1688       30        
cleaning             0.1236       950       
hallway              0.0107       652       
502                  0.0000       138       
511                  0.0000       92        
513                  0.0000       360 